## Урок 4. Система управления базами данных MongoDB в Python

1. Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru. Для парсинга использовать XPath. Структура данных должна содержать:
    * название источника;
    * наименование новости;
    * ссылку на новость;
    * дата публикации.
2. Сложить собранные новости в БД

### Импортирование необходимых библиотек

In [1]:
# Импортируем библиотеку MongoClient из модуля pymongo для работы с MongoDB
from pymongo import MongoClient
import pymongo

# Импортируем класс DuplicateKeyError из модуля pymongo.errors для обработки дубликатов ключей
from pymongo.errors import DuplicateKeyError 

# Импорт библиотеки для работы с JSON-форматом
import json  

# Импорт библиотеки для выполнения HTTP-запросов
import requests  

# Импорт модуля для работы с HTML и XML
from lxml import html 

# Импорт функции для красивого вывода данных в консоль
from pprint import pprint 

# Импорт модуля datetime для работы с датами
from datetime import datetime

# Импортируем библиотеку hashlib для генерации уникальных идентификаторов
import hashlib

1. Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru. Для парсинга использовать XPath. Структура данных должна содержать: 
    * название источника;
    * наименование новости;
    * ссылку на новость;
    * дата публикации.

In [2]:
# Определяем контейнер для json файла
JSON_FILE_PATH = './json_folder/news.json'

### Инициализация сеанса и настройка get-запроса к веб-странице

In [3]:
# Заголовки, которые будут отправлены вместе с запросом
headers = {'User-Agent': 
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/112.0.0.0 Safari/537.36'}

sessions = requests.Session()

# Выполнение HTTP-запроса к указанному URL
response = sessions.get("https://news.mail.ru/", headers=headers)

# Создание объекта HTML-документа из содержимого ответа response.content
dom = html.fromstring(response.content)

# Создание пустого списка для хранения данных о новостях
news = []

# Использование XPath-выражения для выбора элементов новостей из HTML-документа
# В данном случае, выбираются все <li> элементы, содержащие класс 'list__item'
items = dom.xpath("//li[contains(@class, 'list__item')]")

### Подключение к серверу MongoDB и создание коллекции

In [4]:
# Подключение к серверу MongoDB с указанием хоста (127.0.0.1) и порта (27017)
client = MongoClient('127.0.0.1', 27017)

# Получение доступа к базе данных с именем 'news'
db = client['news']

# Удаление существующей коллекции 'news_articles', если она существует
db.drop_collection('news_articles')

# Создание новой коллекции 'news_articles' в базе данных 'news'
news_articles = db.create_collection('news_articles')

### Обработка результата get-запроса

In [5]:
# Обработка каждого элемента с новостью
for item in items:
    news_item = {}
    
    # Извлечение названия новости
    title = item.xpath("normalize-space(translate(.//a[contains(@class, 'list__text')]/text(), '\xa0', ' '))")
    
    # Извлечение ссылки на новость
    link = item.xpath(".//a[contains(@class, 'list__text')]/@href")
    
    # Отправка запроса по ссылке новости
    link_response = requests.get(link[0], headers=headers)
    
    # Проверка успешного получения данных по ссылке
    if link_response.status_code == 200:
        link_tree = html.fromstring(link_response.content)
        
        # Извлечение элементов с датой публикации
        publish_date_elements = link_tree\
            .xpath("//span[contains(@class, 'note__text breadcrumbs__text js-ago')]/@datetime")
        
        # Обработка даты публикации
        if publish_date_elements:
            publish_date = datetime.strptime(publish_date_elements[0], "%Y-%m-%dT%H:%M:%S%z")\
                .strftime("%Y-%m-%d %H:%M:%S")
        else:
            publish_date = "Unknown"
        
        # Извлечение источников новости
        sources = link_tree.xpath("//div[contains(@class, 'article js-article js-module')]")
        sources_list = []

        for element in sources:
            source = element.xpath(".//span[contains(@class, 'link__text')]/text()")[0].strip()
            sources_list.append(source)
        
        # Заполнение полей данных новости
        news_item['title'] = title
        news_item['link'] = link[0]
        news_item['publish_date'] = str(publish_date).split('T')[0]
        
        if sources_list:
            news_item['source'] = sources_list[0]
        else:
            news_item['source'] = "Unknown"
        
        # Создание уникального ID для новости на основе хеширования
        news_item['_id'] = hashlib.sha256(f"{news_item['title']}_{news_item['publish_date']}_{news_item['source']}"\
                                          .encode()).hexdigest()
        
        # Создание уникального индекса на основе хеширования
        news_item['unique_index'] = \
            hashlib.sha256(f"{news_item['link']}_{news_item['publish_date']}_{news_item['source']}"\
                          .encode()).hexdigest()
        
        # Добавление новости в список
        news.append(news_item)
        
        try:
            # Вставка новости в коллекцию и создание индекса
            news_articles.insert_one(news_item)
            news_articles.create_index([('unique_index', pymongo.ASCENDING)], name='unique_id_index', unique=True)

        except pymongo.errors.DuplicateKeyError:
            print(f"Статья с таким id: {news_item.get('_id')} уже существует")
            # Обработка ошибок дубликатов ключей
            pass
        except Exception as e:
            # Обработка других исключений
            print(f"Произошла ошибка: {e}")
    else:
        print(f"Failed to fetch data for link: {link}")

# Сохранение данных в JSON-файл
with open(JSON_FILE_PATH, 'w') as file:
    json.dump(news, file)

# Вывод данных для проверки
print('Данный этап завершен')


Данный этап завершен


### Вывод содержимого коллекции

In [6]:
# Выполняем запрос к базе данных, чтобы получить все документы из коллекции news_articles
articles = list(news_articles.find({}))

# Выводим полученные документы (статьи) в читаемом формате
print("Список статей из базы данных:")
for article in articles:
    print("Заголовок:", article.get("title"))
    print("Ссыдка:", article.get("link"))
    print("Источник:", article.get("source"))
    print("Дата публикации:", article.get("publish_date"))
    print("=" * 30)  # Разделитель между статьями

Список статей из базы данных:
Заголовок: Умер космонавт Александр Викторенко
Ссыдка: https://news.mail.ru/society/57360385/
Источник: © РИА Новости
Дата публикации: 2023-08-10 12:28:40
Заголовок: В США на женщину с неба упала змея, а следом ее атаковал ястреб
Ссыдка: https://news.mail.ru/society/57359571/
Источник: РБК Life
Дата публикации: 2023-08-10 11:57:26
Заголовок: Потепление помогает окуням захватывать озера
Ссыдка: https://news.mail.ru/society/57363163/
Источник: Индикатор
Дата публикации: 2023-08-10 15:02:24
Заголовок: В Португалии косатки оторвали руль яхты на глазах у моряка
Ссыдка: https://news.mail.ru/society/57361439/
Источник: Lenta.Ru
Дата публикации: 2023-08-10 14:14:32
Заголовок: Мошенники начали предлагать клиентам скачать опасное приложение
Ссыдка: https://news.mail.ru/incident/57358651/
Источник: Известия
Дата публикации: 2023-08-10 11:39:59
Заголовок: Ученые выяснили, что ядерная зима будет теплой
Ссыдка: https://pogoda.mail.ru/news/57357877/
Источник: Газета.Ру
Д

In [7]:
# Функция для поиска новостей с фильтром по источнику
def getNews(source, collection):
    try:
        # Формируем запрос (query) для поиска вакансий с минимальной или максимальной зарплатой выше указанной суммы
        query = {
            'source': source   # Поиск новостей по указанному источнику
        }
        # Выполняем запрос к коллекции MongoDB с использованием метода find()
        # и сортируем результаты по полю 'link' в порядке возрастания (1)
        for element in collection.find(query).sort('link', 1):
            # Выводим найденные новости с помощью функции pprint() для более читаемого вывода
            print("Заголовок:", element.get("title"))
            print("Ссыдка:", element.get("link"))
            print("Источник:", element.get("source"))
            print("Дата публикации:", element.get("publish_date"))
            print("=" * 30)  # Разделитель между статьями
    except Exception as e:
        # Обрабатываем исключения, если что-то пошло не так при выполнении запроса
        print(f"Произошла ошибка при чтении файла: {e}")

# Вызов функции с коллекцией news_articles и указанной суммой зарплаты
getNews('Известия', news_articles)

Заголовок: Ведение блога могут признать экономической деятельностью
Ссыдка: https://finance.mail.ru/2023-08-10/vedenie-bloga-mogut-priznat-ekonomicheskoy-deyatelnostyu-57356555/
Источник: Известия
Дата публикации: 2023-08-10 09:27:01
Заголовок: Мошенники начали предлагать клиентам скачать опасное приложение
Ссыдка: https://news.mail.ru/incident/57358651/
Источник: Известия
Дата публикации: 2023-08-10 11:39:59


##### Удаляем данные из коллекции, если необходимо

In [8]:
# Удаляем все документы из коллекции news_articles
deleted_count = news_articles.delete_many({}).deleted_count

# Выводим информацию о количестве удаленных документов
print(f"Удалено {deleted_count} документов из коллекции news_articles.")

Удалено 20 документов из коллекции news_articles.


In [9]:
# Закрываем соединение с базой данных после выполнения операций
client.close()

print("Соединение с базой данных закрыто.")

Соединение с базой данных закрыто.
